## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from itertools import chain

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 2_000_000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 200_000
INITIAL_BALANCE = str(110_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(100_000)
CLAIM_AMOUNT_LIST = [int(round(20_000 * random.random(), -3)) + 20_000 for _ in range(NUMBER_OF_PARTICIPANTS)]

NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
PREFIXES = ['cosmos', 'osmo', 'terra']

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(3)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(2)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf'

INIT_TREASURY_CONTRACT = True
TREASURY_CODE_ID = str(4)
TREASURY_CONTRACT_ADDRESS = 'bostrom18cszlvm6pze0x9sz32qnjq4vtd45xehqs8dq7cwy8yhq35wfnn3q9u647v'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(1)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 2000000
Number of threads: 100


  0%|          | 0/2000000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,28000,john16521126150,QmZ6SJnBFk2b53Gt2ektwsp7ECWP6km2i27UoKybR1q4GX,0x3176ccd5b88d8bf7cabcb3b2f1e6ca4b13b3fc5d,0xd7190e045f87b1287efa6ef1198ebbeecf2e9ba95d63...,bostrom124w2kuuls8c5ezg4fh3au4j2d903a3r58ssn3l,cosmos124w2kuuls8c5ezg4fh3au4j2d903a3r5yryq0c,head maximum daring merit shoulder toast bone ...,bostrom124w2kuuls8c5ezg4fh3au4j2d903a3r58ssn3l...,0x55a59e770124f8a240a6dfe4199167093f32f8c06c93...,{'pub_key': 'A3DPjRBjtIHuj+EleMfBftzEf+Uotgj6s...,eyJwdWJfa2V5IjoiQTNEUGpSQmp0SUh1aitFbGVNZkJmdH...
1,25000,john16521126151,QmYmXFtpG48H5E12zTVs1Le979cQgB6QT8KBdx6t1Wf7ms,0xe6408bb8c8fa21d60bdde3fd9c07f8fe12df61da,0xbf32cafd205fde13b5add27e899e6c36e75a85473b6c...,bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z,cosmos1rdcqvqe3mxah46er4c02pu8fnprz7dm6tp4409,title oval ten able delay target tag orphan go...,bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z...,0xb13e85182b680b7c10a21eac40f6d17a5eb50835b7b0...,{'pub_key': 'A1lHzZScCEr+Hi1QGv/M0qxxjl4B/YEP8...,eyJwdWJfa2V5IjoiQTFsSHpaU2NDRXIrSGkxUUd2L00wcX...
2,31000,john16521126152,QmZDhn45Tup8GnUd4LQHfCtF2Hf9frxXHJ5k7Gm4XK6R2J,0x802e5043a53b8833353bd97e9be85359d05fa5a1,0x5afaa05bffed2385e03a80dd25dd84215c8ee846cbb0...,bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7,osmo1dpu0jwg9x4k528rylsqlsm09rn7ft5ur0knv2t,peanut thing retreat tool pattern limb pet tub...,bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7...,0x3e97255fd5e618a226fbfbd1eb4720447a24e036f054...,{'pub_key': 'A8bfcVcw0FBGkuWqFiLv8GzPnohSt2zPT...,eyJwdWJfa2V5IjoiQThiZmNWY3cwRkJHa3VXcUZpTHY4R3...
3,40000,john16521126153,QmaauStuBTuHUKDveQ5dHsBC4Fo7SZFsgGecXsbwXcoyAh,0x3b1cf67d050e257a1a05eb17f3c346e131cf3895,0x28876dc39babed2bbf212786b3f8adc08195ac967f92...,bostrom1tjqhygt38yhrhkv5scph2ef96ally2y4h7nyev,cosmos1tjqhygt38yhrhkv5scph2ef96ally2y45d8h8t,raise cruise fancy text enrich tower anxiety s...,bostrom1tjqhygt38yhrhkv5scph2ef96ally2y4h7nyev...,0x0cf6b34983d62ad2a3443da27515c232da5c4ed19c41...,{'pub_key': 'AsglEaBaS1Y7UAel9r7T0jPhrEeXfAdI/...,eyJwdWJfa2V5IjoiQXNnbEVhQmFTMVk3VUFlbDlyN1Qwal...
4,33000,john16521126154,QmSAUL7hfidipezWxBG9RqCwZaCxmrkQQyHzeqnGQ6zRJv,0x107481469e6faf29cafc4f7a13265962b83e4667,0x2cf4bf87498e6591a96e2e47fc9035ffba8c86179db6...,bostrom1yu3zs6mfkj92eyxefh7kna27rv42peauvs3fk5,osmo1yu3zs6mfkj92eyxefh7kna27rv42peau8ck27p,seed rebuild sick drive conduct junior soap ca...,bostrom1yu3zs6mfkj92eyxefh7kna27rv42peauvs3fk5...,0x9c4ee685f2c1a0acfd1c758eee4397492d69037a5642...,{'pub_key': 'A9BSJkUEagjdxipS01BCl00tCidT4yJWA...,eyJwdWJfa2V5IjoiQTlCU0prVUVhZ2pkeGlwUzAxQkNsMD...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list_ethereum = \
    [{'address': _item[0],
      'amount': str(_item[1])}
     for _item in claims_df.loc[:, ['ethereum_address', 'amount']].values.tolist()]
root_source_list_cosmos = \
    [{'address': _item[0],
      'amount': str(_item[1])}
     for _item in claims_df.loc[:, ['cosmos_address', 'amount']].values.tolist()]
root_source_list = list(chain.from_iterable(zip(root_source_list_ethereum, root_source_list_cosmos)))

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
root_source_list

[{'address': '0x3176ccd5b88d8bf7cabcb3b2f1e6ca4b13b3fc5d', 'amount': '28000'},
 {'address': 'cosmos124w2kuuls8c5ezg4fh3au4j2d903a3r5yryq0c',
  'amount': '28000'},
 {'address': '0xe6408bb8c8fa21d60bdde3fd9c07f8fe12df61da', 'amount': '25000'},
 {'address': 'cosmos1rdcqvqe3mxah46er4c02pu8fnprz7dm6tp4409',
  'amount': '25000'},
 {'address': '0x802e5043a53b8833353bd97e9be85359d05fa5a1', 'amount': '31000'},
 {'address': 'osmo1dpu0jwg9x4k528rylsqlsm09rn7ft5ur0knv2t', 'amount': '31000'},
 {'address': '0x3b1cf67d050e257a1a05eb17f3c346e131cf3895', 'amount': '40000'},
 {'address': 'cosmos1tjqhygt38yhrhkv5scph2ef96ally2y45d8h8t',
  'amount': '40000'},
 {'address': '0x107481469e6faf29cafc4f7a13265962b83e4667', 'amount': '33000'},
 {'address': 'osmo1yu3zs6mfkj92eyxefh7kna27rv42peau8ck27p', 'amount': '33000'},
 {'address': '0x02de46973caa7617965aed6a94dd105d5d2143b8', 'amount': '37000'},
 {'address': 'osmo108l2v6y2cqhtezff5fnytrkuuf2wyhxr3pg7kv', 'amount': '37000'},
 {'address': '0x723e0d8428401fd69d

In [5]:
NUMBER_OF_THREADS = 30
BASH_SIZE = 2000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_ACTIVATED_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_ACTIVATED_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(f'Number of addresses: {2 * NUMBER_OF_PARTICIPANTS:>,}\nNumber of activated participants: {NUMBER_OF_ACTIVATED_PARTICIPANTS:>,}\nNumber of threads: {NUMBER_OF_THREADS:>,}\nBash size: {BASH_SIZE:>,}\nNumber of tasks: {len(tasks):>,}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

Number of addresses: 4,000,000
Number of activated participants: 200,000
Number of threads: 30
Bash size: 2,000
Number of tasks: 200


  0%|          | 0/200 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_2000000_addresses.json --output temp/proofs_3.json --start_index 6000 --end_index 8001
Merkle root: 07cd81f94d358b0a9e292490e0962de14a55476d2fb41deda78a418d83acf1ae
Number of addresses in the Merkle tree: 2001
Done in 4431.46s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_2000000_addresses.json --output temp/proofs_195.json --start_index 390000 --end_index 392001
Merkle root: 07cd81f94d358b0a9e292490e0962de14a55476d2fb41deda78a418d83acf1ae
Number of addresses in the Merkle tree: 2001
Done in 4541.05s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_2000000_addresses.json --output temp/proofs_82.json --start_index 164000 --end_index 166001
Merkle root: 07cd81f94d358b0a9e292490e0962de14a55476d2fb41deda78a418d83acf1ae
Number of addresses in the Merkle tree: 2001
Done in 4611.90s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_2000000_addresses.json --output temp/proof

In [6]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='inner',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='inner',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,28000,john16521126150,QmZ6SJnBFk2b53Gt2ektwsp7ECWP6km2i27UoKybR1q4GX,0x3176ccd5b88d8bf7cabcb3b2f1e6ca4b13b3fc5d,0xd7190e045f87b1287efa6ef1198ebbeecf2e9ba95d63...,bostrom124w2kuuls8c5ezg4fh3au4j2d903a3r58ssn3l,cosmos124w2kuuls8c5ezg4fh3au4j2d903a3r5yryq0c,head maximum daring merit shoulder toast bone ...,bostrom124w2kuuls8c5ezg4fh3au4j2d903a3r58ssn3l...,0x55a59e770124f8a240a6dfe4199167093f32f8c06c93...,{'pub_key': 'A3DPjRBjtIHuj+EleMfBftzEf+Uotgj6s...,eyJwdWJfa2V5IjoiQTNEUGpSQmp0SUh1aitFbGVNZkJmdH...,[ee71d1332ec65bd98218f7a092f913cc3f6129af546ba...,[c97135be288dfda2f61eaa71ff7d87370d666468c66e5...
1,25000,john16521126151,QmYmXFtpG48H5E12zTVs1Le979cQgB6QT8KBdx6t1Wf7ms,0xe6408bb8c8fa21d60bdde3fd9c07f8fe12df61da,0xbf32cafd205fde13b5add27e899e6c36e75a85473b6c...,bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z,cosmos1rdcqvqe3mxah46er4c02pu8fnprz7dm6tp4409,title oval ten able delay target tag orphan go...,bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z...,0xb13e85182b680b7c10a21eac40f6d17a5eb50835b7b0...,{'pub_key': 'A1lHzZScCEr+Hi1QGv/M0qxxjl4B/YEP8...,eyJwdWJfa2V5IjoiQTFsSHpaU2NDRXIrSGkxUUd2L00wcX...,[a3da1d8fb3e13bc517f0adf1ab26f1acad0173c8032ff...,[41a1393f2d5345e9789ba5e1a46aeaac3e60290ce92e9...
2,31000,john16521126152,QmZDhn45Tup8GnUd4LQHfCtF2Hf9frxXHJ5k7Gm4XK6R2J,0x802e5043a53b8833353bd97e9be85359d05fa5a1,0x5afaa05bffed2385e03a80dd25dd84215c8ee846cbb0...,bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7,osmo1dpu0jwg9x4k528rylsqlsm09rn7ft5ur0knv2t,peanut thing retreat tool pattern limb pet tub...,bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7...,0x3e97255fd5e618a226fbfbd1eb4720447a24e036f054...,{'pub_key': 'A8bfcVcw0FBGkuWqFiLv8GzPnohSt2zPT...,eyJwdWJfa2V5IjoiQThiZmNWY3cwRkJHa3VXcUZpTHY4R3...,[c91e3465b66a0f8908c29e329f105de3370057e110a3f...,[e674ee80ddbad3ab382d41f309af880cea888ec148884...
3,40000,john16521126153,QmaauStuBTuHUKDveQ5dHsBC4Fo7SZFsgGecXsbwXcoyAh,0x3b1cf67d050e257a1a05eb17f3c346e131cf3895,0x28876dc39babed2bbf212786b3f8adc08195ac967f92...,bostrom1tjqhygt38yhrhkv5scph2ef96ally2y4h7nyev,cosmos1tjqhygt38yhrhkv5scph2ef96ally2y45d8h8t,raise cruise fancy text enrich tower anxiety s...,bostrom1tjqhygt38yhrhkv5scph2ef96ally2y4h7nyev...,0x0cf6b34983d62ad2a3443da27515c232da5c4ed19c41...,{'pub_key': 'AsglEaBaS1Y7UAel9r7T0jPhrEeXfAdI/...,eyJwdWJfa2V5IjoiQXNnbEVhQmFTMVk3VUFlbDlyN1Qwal...,[1ba325a63aa614d21335e935685391d5e8d15e9cdbd1c...,[70f8ea11fc54463b6d1d04e023ea7af8ce86afb494343...
4,33000,john16521126154,QmSAUL7hfidipezWxBG9RqCwZaCxmrkQQyHzeqnGQ6zRJv,0x107481469e6faf29cafc4f7a13265962b83e4667,0x2cf4bf87498e6591a96e2e47fc9035ffba8c86179db6...,bostrom1yu3zs6mfkj92eyxefh7kna27rv42peauvs3fk5,osmo1yu3zs6mfkj92eyxefh7kna27rv42peau8ck27p,seed rebuild sick drive conduct junior soap ca...,bostrom1yu3zs6mfkj92eyxefh7kna27rv42peauvs3fk5...,0x9c4ee685f2c1a0acfd1c758eee4397492d69037a5642...,{'pub_key': 'A9BSJkUEagjdxipS01BCl00tCidT4yJWA...,eyJwdWJfa2V5IjoiQTlCU0prVUVhZ2pkeGlwUzAxQkNsMD...,[545a98673c22a235c765f77deacb9abd35bf39b226548...,[28f4284c85cf51975520750d286fe29135f8d3a106bce...


In [7]:
for i in range(ceil(NUMBER_OF_ACTIVATED_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [8]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza
Avatar subgraph contract address: bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf
Proof subgraph contract address: bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp


### Instantiate Passport Contract

In [9]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf


### Set executor in the Subgraph Contracts

In [10]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Treasury Contract

In [11]:
if INIT_TREASURY_CONTRACT:
    treasury_contract_address = \
        instantiate_contract(
            init_query=f'''{{"admins": ["{WALLET_ADDRESS}"], "mutable": true}}''',
            contract_code_id=TREASURY_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test treasury',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    treasury_contract_address = TREASURY_CONTRACT_ADDRESS
print(f'Treasury contract address: {treasury_contract_address}')

Treasury contract address: bostrom1nc84knc0n7td5xqplwy0luh97zd8hv5mhvm9cdempc05xk0xvxyqvwhyll


### Instantiate Gift Contract

In [12]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "treasury":"{treasury_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1l6ut7ymdzhf38wldp0eu6ccjawgvzku69kmjl6ddjqpcm9f62gkqufluv3


### Initiate Class of Output Parsing

In [13]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   treasury_contract_address: 'Treasury Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Set Allowance in Treasury contract

In [14]:
contract_utils.parse_contract_execution_json(
    contract_utils.send_coins(
        from_seed=WALLET_SEED,
        to_addresses=[treasury_contract_address],
        amounts=[INITIAL_BALANCE],
        gas=min(100_000 + 25_000, int(20e6)),
        display_data=DISPLAY_TX_EXECUTION))


Events

coin received
	receiver: Treasury Contract
	amount: 110000000000boot

coin spent
	spender: Passport Owner Address
	amount: 110000000000boot

message from bank bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0 /cosmos.bank.v1beta1.MsgMultiSend

transfer
	recipient: Treasury Contract
	amount: 110000000000boot
Gas used: 73,369
Tx hash: A13F05CE7E0BEA7F738E3FDD168C14DA6316825E2DC768FD0F6425F4C47C9C2B


In [15]:
def set_allowance(amount: str = INITIAL_BALANCE, display_data: bool = False):
    return contract_utils.execute_contract(
        execute_msg={"increase_allowance": {"spender": gift_contract_address, "amount": {"amount": amount, "denom": 'boot'}}},
        contract_address=treasury_contract_address,
        mnemonic=WALLET_SEED,
        gas=600000,
        display_data=display_data)

contract_utils.parse_contract_execution_json(
    set_allowance(display_data=False))


Events

execute
	execute contract: Treasury Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Treasury Contract
	action: increase_allowance
	owner: Passport Owner Address
	spender: Gift Contract
	denomination: boot
	amount: 110,000,000,000
Gas used: 135,441
Tx hash: 70C18E706A173AEE762262EDA659716FDB9BB86F6D23F4CC9B85FDB4AE47A6BC


### Register Merkle Root

In [16]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 07cd81f94d358b0a9e292490e0962de14a55476d2fb41deda78a418d83acf1ae
Gas used: 131,276
Tx hash: 84C44411CF66B40A57C71A8920E85DA9A46C382BD72AB16982A4618BA1049E8D


#### Get Merkle Root form the Gift Contract

In [17]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1l6ut7ymdzhf38wldp0eu6ccjawgvzku69kmjl6ddjqpcm9f62gkqufluv3
{'data': {'merkle_root': '07cd81f94d358b0a9e292490e0962de14a55476d2fb41deda78a418d83acf1ae'}}


### Send coins to new addresses

In [21]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 900

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

send_output = []
for bostrom_addresses_item in tqdm(bostrom_addresses_chunks):
    try:
        send_output.append(
            contract_utils.send_coins(
                from_seed=WALLET_SEED,
                to_addresses=bostrom_addresses_item,
                amounts=[1] * len(bostrom_addresses_item),
                gas=min(150_000 + 30_000 * len(bostrom_addresses_item), int(24e6)),
                display_data=DISPLAY_TX_EXECUTION))
    except Exception as e:
        print(f'Error: {e}')

  0%|          | 0/223 [00:00<?, ?it/s]

In [22]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output[0])


Events

coin received
	receiver: bostrom124w2kuuls8c5ezg4fh3au4j2d903a3r58ssn3l
	amount: 1boot
	receiver: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	amount: 1boot
	receiver: bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7
	amount: 1boot
	receiver: bostrom1tjqhygt38yhrhkv5scph2ef96ally2y4h7nyev
	amount: 1boot
	receiver: bostrom1yu3zs6mfkj92eyxefh7kna27rv42peauvs3fk5
	amount: 1boot
	receiver: bostrom108l2v6y2cqhtezff5fnytrkuuf2wyhxr6f0a7e
	amount: 1boot
	receiver: bostrom1jg7raggv8lmmg5f3z9s8jnwarz36t4e7k478gr
	amount: 1boot
	receiver: bostrom12cqstlmdtmcs8xt9fpzk5z0y2wpe69kr3494ez
	amount: 1boot
	receiver: bostrom1ers3rza08vpkpe0c2alzfsla24qqe4egr6qnmq
	amount: 1boot
	receiver: bostrom1urllwdtekqk497hslza5ss2a8uefay8jf4vf9v
	amount: 1boot
	receiver: bostrom1rwssvf6x9dk399gu8na9xx6wz4t0h9ez39nwpg
	amount: 1boot
	receiver: bostrom1ym0rlf63p4sjy8qwv6ypnc6apw4stnnlhqvqcn
	amount: 1boot
	receiver: bostrom1eajyse3nhlv8g5hutdz8qz3v8ef6m7ksafzamu
	amount: 1boot
	receiver: bostrom1tw4zara55

### Create Passports, Proof Addresses and Claim Gift

In [23]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 200,000
Number of threads: 100:>,


  0%|          | 0/192000 [00:00<?, ?it/s]

python3 create_passport_and_claim_job.py temp/6_participants_test_data_2000000_addresses.csv 60944 bostrom1l6ut7ymdzhf38wldp0eu6ccjawgvzku69kmjl6ddjqpcm9f62gkqufluv3
b'Traceback (most recent call last):\n  File "/home/grovybear/cyber/cw-cybergift/contracts/cw-cyber-airdrop/testdata/generate_test_data/create_passport_and_claim_job.py", line 67, in <module>\n    res = participation(row=row, address_dict=address_dict, release_bool=release_bool)\n  File "/home/grovybear/cyber/cw-cybergift/contracts/cw-cyber-airdrop/testdata/generate_test_data/create_passport_and_claim_job.py", line 34, in participation\n    _claim_ethereum_json = contract_utils.claim(row, display_data=DISPLAY_TX_EXECUTION)\n  File "/home/grovybear/cyber/cw-cybergift/contracts/cw-cyber-airdrop/testdata/generate_test_data/contract_utils.py", line 206, in claim\n    return self.execute_contract(\n  File "/home/grovybear/cyber/cw-cybergift/contracts/cw-cyber-airdrop/testdata/generate_test_data/contract_utils.py", line 185, in 

In [41]:
with open('temp/contract_participation_execution_log_10000.txt', 'r') as f:
    data = json.load(f)

In [42]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	owner: bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2
	token_id: 319041
Gas used: 342,370
Tx hash: FE9B54C9E5C895D9353312B8BBF0CE012D7757B10F28CD3899EEDADDDEBCCBD9


In [43]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16521126159991
	address: 0xdc431e04a2ce31e90bfbd6382d81b14ee8cb4059
Gas used: 237,523
Tx hash: 8156E96D4BB51547B7C0E5ED5917129E52323AAD4C7653A86F02D3F18C662F89


In [44]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16521126159991
	address: osmo136zz7jck89ug4r0g9wxdnp97h26ul6xyny7nnl
Gas used: 240,855
Tx hash: 555AFEB1DC5F13D78678BAA8FF4C508B7404888D2FB5F62702B561D15BD1F7BC


In [45]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2
	amount: 100000boot

coin spent
	spender: Treasury Contract
	amount: 100000boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2
	sender: Treasury Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: osmo136zz7jck89ug4r0g9wxdnp97h26ul6xyny7nnl
	target: bostrom136zz7jck89ug4r0g9wxdnp97h26ul6xycvesm2
	amount: 301,969
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 401,953
Tx hash: ECEB867CA0374AF777B5333384F4EA26FC6EBA13C6B557B7D808E442C84BF983


### Release Gift

In [29]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 200000
Number of threads: 100


  0%|          | 0/200000 [00:00<?, ?it/s]

In [30]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [31]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	amount: 22496boot

coin spent
	spender: Treasury Contract
	amount: 22496boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	sender: Treasury Contract
	amount: 22496boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	gift_address: 0xe6408bb8c8fa21d60bdde3fd9c07f8fe12df61da
	stage: 9
	amount: 22,496
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,533
Tx hash: DC96F11362215BCA80C34CBD0597124C6AAD7836D0D631183773B4BBC75BA68E


In [32]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	amount: 22491boot

coin spent
	spender: Treasury Contract
	amount: 22491boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	sender: Treasury Contract
	amount: 22491boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	gift_address: cosmos1rdcqvqe3mxah46er4c02pu8fnprz7dm6tp4409
	stage: 9
	amount: 22,491
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,778
Tx hash: 3764AA0D04C112D0ABFE8582AD36190CEA618ECC972091E267114DE9E1614D2F


## Passport NFT testing
### Transfer Passport

In [33]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [34]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom1rdcqvqe3mxah46er4c02pu8fnprz7dm6gjpx3z
	recipient: Passport Owner Address
	token_id: 309120
Gas used: 424,079
Tx hash: FAD94B4EE42D8BE78DC4391CB7D062DF281A2527F123700BED9294A61A97C90E


### Burn

In [35]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7 wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1dpu0jwg9x4k528rylsqlsm09rn7ft5ury750z7
	token_id: 309055
Gas used: 382,926
Tx hash: 1F7865D1A3241500C9D65B9C916C2F74C08692063D055FFD709545CEDB5E8F62


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +